In [552]:
import yaml
import numpy as np
import numpy.random as rnd

In [553]:
document = """
---
columns : [host_id, time, type, offset, len]
pattern : "{:>7}|{:>5}|{:>5}|{:>9}|{:>3}"
max_reqs : 100
max_offset : 1000
output : "help.txt"
---
host_id : 1
host_prob : 0.5
new_offset_prob : [0.5, 0.5]
mem_split : [0.8, 0.2]
mem_prob : [0.5, 0.5]
rw_prob : [0.5, 0.5]
req_len : [10, 20, 30]
req_len_prob : [0.25, 0.25, 0.5]
req_wait : [1, 5, 10, 25]
req_wait_prob : [0.25, 0.25, 0.25, 0.25]
---
host_id : 2
host_prob : 0.5
new_offset_prob : [0.99, 0.01]
mem_split : [0.8, 0.2]
mem_prob : [0.5, 0.5]
rw_prob : [0.5, 0.5]
req_len : [10, 20, 30]
req_len_prob : [0.25, 0.25, 0.5]
req_wait : [1, 13, 17, 21]
req_wait_prob : [0.25, 0.25, 0.25, 0.25]
...
"""

In [554]:
class host:
    
    def __init__(self, info):
        self.__dict__ = info
        
        for key, item in self.__dict__.items():
            self.__dict__[key] = np.array(item) if type(item) is list else item
        
        self.curr_offset = self.max_offset
    
    def gen_req(self):
        self.gen_req_type()
        self.gen_req_len()
        self.gen_req_wait_time()
        if self.new_offset() or self.curr_offset + self.curr_req_len > self.max_offset:
            self.gen_offset()
            print('oopsie_1 : ', 'host {0}, offset {1}, len {2}'.format(self.host_id, self.curr_offset, self.curr_req_len))
            while self.curr_offset + self.curr_req_len > self.max_offset:
                print('WHOOPS')
                self.gen_offset()
        
    def new_offset(self):
        return rnd.choice([0,1], p=self.new_offset_prob)
        
    def gen_req_type(self):
        self.req_type = rnd.choice(['READ', 'WRITE'], p=self.rw_prob)
            
    def gen_req_len(self):
        self.curr_req_len = rnd.choice(self.req_len, p=self.req_len_prob)
        
    def gen_req_wait_time(self):
        self.req_wait_time = rnd.choice(self.req_wait, p=self.req_wait_prob)
    
    def gen_offset(self):
        tmp = rnd.choice([0,1], p=self.mem_prob)
        self.curr_offset = rnd.randint(int(self.max_offset * self.mem_split[tmp])) +\
                                                int(self.max_offset * self.mem_split[0]) * tmp
        
    def send_req(self):
        self.gen_req()
        self.curr_offset += self.curr_req_len
        return [self.host_id, self.req_wait_time, self.req_type, (self.curr_offset - self.curr_req_len), self.curr_req_len]
    

In [555]:
class host_handler:
    def __init__(self, cfg_file):
        cfg = [i for i in yaml.safe_load_all(cfg_file)]
        self.__dict__ = cfg[0]
        self.out = open(self.output, 'w')
        del cfg[0]
        self.active_hosts = []
        self.host_prob = []
        for i in cfg:
            i['max_offset'] = self.max_offset
            tmp_host = host(i)
            self.active_hosts.append(tmp_host)
            self.host_prob.append(tmp_host.host_prob)
        self.host_prob = np.array(self.host_prob)
        self.out.write(self.pattern.format(*self.columns) + '\n')
        
    def gen_trace(self):
        curr_time = 0
        for i in range(self.max_reqs):
            req = rnd.choice(self.active_hosts, p=self.host_prob).send_req()
            curr_time += req[1]
            req[1] = curr_time
            self.out.write(self.pattern.format(*req) + '\n')
        self.out.close()

In [556]:
t = host_handler(document)
t.gen_trace()

oopsie_1 :  host 1, offset 98, len 10
oopsie_1 :  host 2, offset 853, len 10
oopsie_1 :  host 1, offset 791, len 10
oopsie_1 :  host 1, offset 700, len 30
oopsie_1 :  host 2, offset 656, len 30
oopsie_1 :  host 1, offset 922, len 30
oopsie_1 :  host 1, offset 326, len 30
oopsie_1 :  host 1, offset 129, len 10
oopsie_1 :  host 1, offset 538, len 30
oopsie_1 :  host 1, offset 337, len 10
oopsie_1 :  host 1, offset 883, len 20
oopsie_1 :  host 1, offset 927, len 20
oopsie_1 :  host 1, offset 357, len 20
oopsie_1 :  host 1, offset 940, len 30
oopsie_1 :  host 1, offset 849, len 10
oopsie_1 :  host 2, offset 979, len 30
WHOOPS
oopsie_1 :  host 1, offset 911, len 10
oopsie_1 :  host 1, offset 722, len 30
oopsie_1 :  host 1, offset 623, len 20
oopsie_1 :  host 1, offset 425, len 30
oopsie_1 :  host 1, offset 74, len 30
oopsie_1 :  host 1, offset 867, len 10
oopsie_1 :  host 1, offset 4, len 30
oopsie_1 :  host 1, offset 923, len 10
oopsie_1 :  host 1, offset 49, len 20
oopsie_1 :  host 2, off